In [148]:
import pandas as pd
import re

In [149]:
REQS_FILE = 'CU_SR_OPEN_DATA_CATALOG'
REQS_FILE_ENCODING = 'utf-16le'

In [150]:
reqs: pd.DataFrame = pd.read_csv(f"data/{REQS_FILE}.csv", encoding=REQS_FILE_ENCODING)
reqs.head(1)

,Course ID,Subject,Catalog,Long Title,Class Units,Component Code,Component Descr,Pre Requisite Description,Career,Equivalent Courses
0,26,ACCO,220,Financial and Managerial Accounting,3.0,LEC,Lecture,"Never Taken/Not Registered: ACCO213, ACCO21...",UGRD,NaN


In [154]:
incorrect_subject_regex = fr"({"|".join(reqs["Subject"].unique())})"
incorrect_subject_pattern = re.compile(fr"{incorrect_subject_regex}[^0-9]*(\d+)")
def standardize_subject_codes(entry: str) -> str:
        return incorrect_subject_pattern.sub(r"\1\2", entry)

def standardize_terms(entry: str) -> str:
    # Subject codes 
    entry = standardize_subject_codes(entry)

    # Term prerequisite
    entry = entry.replace("prerequisite", "Prerequisite")
    entry = entry.replace("The following courses must be completed previously", "Prerequisite")
    entry = entry.replace("PREREQ", "Prerequisite")
    entry = entry.replace("Pre-requisite", "Prerequisite")
    entry = entry.replace("Course Prerequisite", "Prerequisite")

    # # Term corequisite
    entry = entry.replace("corequisite", "Corequisite")
    entry = entry.replace("concurrently", "Corequisite")
    entry = entry.replace("Co-requisite", "Corequisite")
    entry = entry.replace("Co-currently", "Corequisite")
    entry = entry.replace("Course Corequisite", "Corequisite")

    entry = entry.replace("Previously or Corequisite", "Prerequisite/Corequisite")
    # Prefer these representations
    entry = re.sub(r"You must complete 1 of the following rules[.]?",
                   r"At least one of",
                   entry)
    return entry

def remove_unnecessary_text(entry: str) -> str:
    # These are global rules, so they don't need to be explicitely written
    entry = re.sub(r"Must complete all 200 level courses before enrolling in[\s]*(any)?[\s]*400 level course(s.)?",
                   r"",
                   entry,
                   flags=re.IGNORECASE)
    entry = re.sub(r"Students must complete all 200-level courses required for their program before registering for any 400-level courses.",
                   r"",
                   entry,
                   flags=re.IGNORECASE)
    return entry

def trim_whitespace(entry: str) -> str:
    return entry.strip()

def consolidate_redundant_whitespace(entry: str) -> str:
    return re.sub(r"\s+", " ", entry)

In [155]:
cleaned_reqs: pd.DataFrame = reqs.copy()
cleaned_reqs.fillna("None", inplace=True)

# Replace NA values with empty strings
cleaned_reqs['Pre Requisite Description'] = \
    cleaned_reqs['Pre Requisite Description'].\
        apply(remove_unnecessary_text).\
        apply(standardize_subject_codes).\
        apply(standardize_terms).\
        apply(trim_whitespace).\
        apply(consolidate_redundant_whitespace)

cleaned_reqs.head(3)

,Course ID,Subject,Catalog,Long Title,Class Units,Component Code,Component Descr,Pre Requisite Description,Career,Equivalent Courses
0,26,ACCO,220,Financial and Managerial Accounting,3.0,LEC,Lecture,"Never Taken/Not Registered: ACCO213, ACCO218, ...",UGRD,None
1,27,ACCO,230,Introduction to Financial Accounting,3.0,LEC,Lecture,"Never Taken/Not Registered: ACCO213, ACCO220, ...",UGRD,None
2,28,ACCO,240,Introduction to Managerial Accounting,3.0,LEC,Lecture,"Never Taken/Not Registered: ACCO218, ACCO220, ...",UGRD,None


In [156]:
cleaned_reqs.to_csv(f"data/{REQS_FILE}_cleaned.csv", index=False)